# LAB 2

## Unigrams, Bigrams, and Trigrams in Naive Bayes Classifiers ##


3. Use the data in Tables 13.1  and 13.10  from [1] and the Naive Bayes for Spam Detection from IAML Edimburg with the N Lidell code

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
#Reading TABLE 13.1
df = pd.read_csv('./Tabla131.csv', usecols=[0,1], encoding='latin-1')
df.columns = ['label','body']
# label spam as 1, not spam as 0
df['label'] = df['label'].replace(["no","yes"],[0,1])
data1 = df.values
df.head()
print("*****TABLE 13.10*****")
print(data1)

#Reading TABLE 13.10
df= pd.read_csv('./Tabla1310.csv', usecols=[0,1], encoding='latin-1')
df.columns = ['label','body']
#label spam as 1, not spam as 0
df['label'] = df['label'].replace(["no","yes"],[0,1])
data10 = df.values
print("*****TABLE 13.10*****")
print(data10)

#Reading TABLE IAM
df= pd.read_csv('./IAM.csv', usecols=[0,1], encoding='latin-1')
df.columns = ['label','body']
#label spam as 1, not spam as 0
df['label'] = df['label'].replace(["ham","spam"],[0,1])
data2 = df.values
print("*****TABLE IAM*****")
print(data2)

*****TABLE 13.10*****
[[1 'Chinese Beijing Chinese']
 [1 'Chinese Chinese Shanghai']
 [1 'Chinese Macao']
 [0 'Tokyo Japan Chinese']
 [1 'Chinese Chinese Chinese Tokyo Japan']]
*****TABLE 13.10*****
[[1 'Taipei Taiwan']
 [1 'Macao Taiwan Shanghai']
 [0 'Japan Sapporo']
 [0 'Sapporo Osaka Taiwan']
 [0 'Taiwan Taiwan Sapporo']]
*****TABLE IAM*****
[[1 'send us your password']
 [0 'send us your review']
 [0 'review your password']
 [1 'send us']
 [1 'send us your password']
 [1 'send us your account']
 [0 'review us now']]


In [3]:
class ngrams_bayes():
    
    def __init__(self, data, n=2, split=0.8):
        
        # split into training and testing data
        self.train_data, self.test_data = train_test_split(data,
                                                          train_size=split, shuffle=False) 
        # convert into n grams
        self.train_data = [[item[0], self.ngrams(n, item[1])] for item in self.train_data]
        self.test_data = [[item[0], self.ngrams(n, item[1])] for item in self.test_data]
    
        
        # count unique n grams in training data
        flattened = [gram for message in self.train_data for gram in message[1]]
        self.unique = len(set(flattened))
        
        # init dicts
        self.trainPositive = {}
        self.trainNegative = {}
        # counters
        self.posGramCount = 0
        self.negGramCount = 0
        self.spamCount = 0
        # priors
        self.pA = 0
        self.pNotA = 0
        
        
    def ngrams(self, n, text):
        text = text.split(' ')
        grams = []
        for i in range(len(text)-n+1):
            gram = ' '.join(text[i:i+n])
            grams.append(gram)
        return grams 
    
    def train(self):
        
        for item in self.train_data:
            label = item[0]
            grams = item[1]
            if label == 1:
                self.spamCount += 1   
            for gram in grams:
                if label == 1:
                    self.trainPositive[gram] = self.trainPositive.get(gram, 0) + 1
                    self.posGramCount += 1
                else:
                    self.trainNegative[gram] = self.trainNegative.get(gram, 0) + 1
                    self.negGramCount += 1
                    
        #probabilidad de A Y DE A COMPLEMENTO             
        self.pA = self.spamCount/float(len(self.train_data))
        self.pNotA = 1.0 - self.pA
         
    def classify(self, text, alpha=1.0):
        
        self.alpha = alpha
        isSpam = self.pA * self.conditionalText(text, 1)
        notSpam = self.pNotA * self.conditionalText(text, 0)
        print("is spam probability: " + str(isSpam))
        print("not spam probability: " + str(notSpam))
        if (isSpam > notSpam):
            return 1
        else:
            return 0
        
    def conditionalText(self, grams, label):
        result = 1.0
        for ngram in grams:
            result *= self.conditionalNgram(ngram, label)
        return result
    
    def conditionalNgram(self, ngram, label):
        alpha = self.alpha
        if label == 1:
            return ((self.trainPositive.get(ngram,0)+alpha) /
                    float(self.posGramCount+alpha*self.unique))
        else:
            return ((self.trainNegative.get(ngram,0)+alpha) /
                    float(self.negGramCount+alpha*self.unique))
            
    def evaluate_test_data(self):
        results = []
        for test in self.test_data:
            label = test[0]
            text = test[1]
            ruling = self.classify(text)
            if ruling == label:
                results.append(1) 
            else:
                results.append(0) 
        
        
        print("Evaluated {} test cases. {:.2f}% Accuracy".format(len(results), 100.0*sum(results)/float(len(results))))
        return sum(results)/float(len(results))

In [8]:
#UNIGRAMS FOR THE DATA
import warnings
warnings.filterwarnings("ignore")
unigram_bayes1 = ngrams_bayes(data1,1)
unigram_bayes10 = ngrams_bayes(data10,1)
unigram_bayes2 = ngrams_bayes(data2,split=0.9)
unigram_bayes1.train()
unigram_bayes10.train()
unigram_bayes2.train()

print("***The test set for the Table 13.1 is:***")
print(unigram_bayes1.test_data)
print("***The test set for the Table 13.10 is:***")
print(unigram_bayes10.test_data)
print("***The test set for IAM is:***")
print(unigram_bayes2.test_data)

***The test set for the Table 13.1 is:***
[[1, ['Chinese', 'Chinese', 'Chinese', 'Tokyo', 'Japan']]]
***The test set for the Table 13.10 is:***
[[0, ['Taiwan', 'Taiwan', 'Sapporo']]]
***The test set for IAM is:***
[[0, ['review us', 'us now']]]


In [9]:
print("*****RESULTS FOR THE TABLE 13.1*****")
unigram_bayes1.evaluate_test_data()
print("*****RESULTS FOR THE TABLE 13.10*****")
unigram_bayes10.evaluate_test_data()
print("*****RESULTS FOR IAM*****")
unigram_bayes2.evaluate_test_data()

*****RESULTS FOR THE TABLE 13.1*****
is spam probability: 0.00030121377997263036
not spam probability: 0.00013548070246744226
Evaluated 1 test cases. 100.00% Accuracy
*****RESULTS FOR THE TABLE 13.10*****
is spam probability: 0.0026041666666666665
not spam probability: 0.003472222222222222
Evaluated 1 test cases. 100.00% Accuracy
*****RESULTS FOR IAM*****
is spam probability: 0.0026041666666666665
not spam probability: 0.002754820936639119
Evaluated 1 test cases. 100.00% Accuracy


1.0

In [10]:
#Bigrams tabla 13.1 
print("***********RESULTS FOR THE TABLE 13.1***********")
bigram_sms1= ngrams_bayes(data1,2) 
print(bigram_sms1)
bigram_sms1.train()
bigram_sms1.evaluate_test_data()
#Bigrams tabla 13.10
print("***********RESULTS FOR THE TABLE 13.10***********")
bigram_sms10= ngrams_bayes(data10,2) 
print(bigram_sms10)
bigram_sms10.train()
bigram_sms10.evaluate_test_data()

#Bigrams tabla IAM
print("***********RESULTS FOR IAM***********")
bigram_sms2= ngrams_bayes(data2,2) 
print(bigram_sms2)
bigram_sms2.train()
bigram_sms2.evaluate_test_data()

***********RESULTS FOR THE TABLE 13.1***********
is spam probability: 0.00014467592592592592
not spam probability: 7.620789513793628e-05
Evaluated 1 test cases. 100.00% Accuracy
***********RESULTS FOR THE TABLE 13.10***********
is spam probability: 0.006172839506172839
not spam probability: 0.006172839506172839
Evaluated 1 test cases. 100.00% Accuracy
***********RESULTS FOR IAM***********
is spam probability: 0.004166666666666667
not spam probability: 0.0016000000000000005
is spam probability: 0.004166666666666667
not spam probability: 0.004000000000000001
Evaluated 2 test cases. 50.00% Accuracy


0.5

In [11]:
#Trigrams table 13.1
print("***********RESULTS FOR THE TABLE 13.1***********")
trigram_sms1 = ngrams_bayes(data1,3) 
trigram_sms1.train()
trigram_sms1.evaluate_test_data()

#Trigrams table 13.10
print("***********RESULTS FOR THE TABLE 13.10***********")
trigram_sms10 = ngrams_bayes(data10,3) 
trigram_sms10.train()
trigram_sms10.evaluate_test_data()

#Trigrams tabla IAM
print("***********RESULTS FOR IAM***********")
trigram_sms2 = ngrams_bayes(data2,3) 
trigram_sms2.train()
trigram_sms2.evaluate_test_data()

***********RESULTS FOR THE TABLE 13.1***********
is spam probability: 0.006000000000000002
not spam probability: 0.00390625
Evaluated 1 test cases. 100.00% Accuracy
***********RESULTS FOR THE TABLE 13.10***********
is spam probability: 0.16666666666666666
not spam probability: 0.16666666666666666
Evaluated 1 test cases. 100.00% Accuracy
***********RESULTS FOR IAM***********
is spam probability: 0.028124999999999997
not spam probability: 0.016326530612244896
is spam probability: 0.075
not spam probability: 0.05714285714285714
Evaluated 2 test cases. 50.00% Accuracy


0.5